In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
archivo = "dataset/Estadisticas_historicas_comercializacion.xlsx"

# Cargar datos
df = pd.read_excel(archivo)

df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Estadisticas_historicas_comercializacion.xlsx'

Bien logramos identificar las columnas importantes ahora vamos a solo a agarrar las que nos interesan. 

In [5]:
df_importacion = df_importacion[['Fecha', 'Gasolina regular', 'Gasolina superior', 'Diesel alto azufre']]
df_consumo = df_consumo[['Fecha', 'Gasolina regular', 'Gasolina superior', 'Diesel alto azufre']]


Vamos a analizar como viene formateada la fecha.. 

In [6]:
print(df_importacion['Fecha'].head())

0    2001-01-01 00:00:00
1    2001-02-01 00:00:00
2    2001-03-01 00:00:00
3    2001-04-01 00:00:00
4    2001-05-01 00:00:00
Name: Fecha, dtype: object


MMm es tipo object asi que la vamos a transformar a datetime

In [7]:
df_importacion['Fecha'] = pd.to_datetime(df_importacion['Fecha'])
df_consumo['Fecha'] = pd.to_datetime(df_consumo['Fecha'])

DateParseError: Unknown datetime string format, unable to parse: Fuente: informes mensuales de titulares de licencias de la cadena de comercialización de hidrocarburos., at position 289